# PADUFES20



Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import sys
import time
import gc
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import copy
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

from efficientnet_pytorch import EfficientNet
import torchextractor as tx

In [2]:
sys.path.append('..')

from utils.train import train
from utils.metrics import get_scores, get_metrics
from utils.dataset import get_data_loader

# Models

In [3]:
def get_model(model_name, n_classes=8, pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model      = None
    input_size = 0
    weights    = None

    if 'resnet' in model_name:
        model_n = model_name[6:]
        if pretrained:
            weights = f'ResNet{model_n}_Weights.DEFAULT'

        if model_name == 'resnet18':
            model = models.resnet18(weights=weights)
        elif model_name == 'resnet34':
            model = models.resnet34(weights=weights)
        elif model_name == 'resnet50':
            model = models.resnet50(weights=weights)
        elif model_name == 'resnet101':
            model = models.resnet101(weights=weights)
        elif model_name == 'resnet152':
            model = models.resnet152(weights=weights)
        else:
            raise Exception('Resnet model must be resnet18, resnet34, resnet50, resnet101 or resnet152')

        n_feats = model.fc.in_features
        input_size     = 224
        model.fc       = nn.Linear(n_feats, n_classes)
        feats_layer    = -4

    elif 'resnext' in model_name:
        if pretrained:
            weights = f'DEFAULT'

        if model_name == 'resnext50':
            model = models.resnext50_32x4d(weights=weights)
        elif model_name == 'resnext101':
            model = models.resnext101_32x8d(weights=weights)

        n_feats = model.fc.in_features
        input_size     = 224
        model.fc       = nn.Linear(n_feats, n_classes)
        feats_layer    = -4
        

    elif 'effnet' in model_name:
        model_n = model_name[-1]
        full_model_name = f'efficientnet-b{model_n}'

        if pretrained:
            model = EfficientNet.from_pretrained(full_model_name, num_classes=n_classes)
        else:
            model = EfficientNet.from_name(full_model_name, num_classes=n_classes)

        n_feats        = model._fc.in_features
        input_size     = EfficientNet.get_image_size(full_model_name)
        feats_layer    = None


    elif 'vgg' in model_name:
        if pretrained:
            weights = 'DEFAULT'

        if model_name == 'vgg11':
            model = models.vgg11(weights=weights)
        elif model_name == 'vgg13':
            model = models.vgg13(weights=weights)
        elif model_name == 'vgg16':
            model = models.vgg16(weights=weights)
        elif model_name == 'vgg19':
            model = models.vgg19(weights=weights)

        n_feats             = 512
        n_class_feats       = model.classifier[6].in_features

        input_size          = 224
        model.classifier[6] = nn.Linear(n_class_feats, n_classes)
        feats_layer = -9

    elif model_name == 'alexnet':
        if pretrained:
            weights = 'AlexNet_Weights.DEFAULT'
        model = models.alexnet(weights=weights)

        # n_feats             = model.classifier[6].in_features
        n_feats             = 256
        n_class_feats       = model.classifier[6].in_features

        input_size          = 224
        model.classifier[6] = nn.Linear(n_class_feats, n_classes)
        feats_layer = -10

    elif 'vit' in model_name:
        vit_type = model_name[4]
        vit_num  = model_name[6:8]
        if pretrained:
            weights = f'ViT_{vit_type.upper()}_{vit_num}_Weights.IMAGENET1K_V1'

        if model_name == 'vit_b_16':
            model = models.vit_b_16(weights=weights)
        elif model_name == 'vit_b_32':
            model = models.vit_b_32(weights=weights)
        elif model_name == 'vit_l_16':
            model = models.vit_l_16(weights=weights)
        elif model_name == 'vit_l_32':
            model = models.vit_l_32(weights=weights)
        elif model_name == 'vit_h_14':
            weights = 'DEFAULT'
            model = models.vit_h_14(weights=weights)
        else:
            raise Exception('ViT model must be vit_b_16, vit_b_32, vit_l_16, vit_l_32, or vit_h_14')
            
        n_feats          = model.hidden_dim
        model.heads.head = nn.Linear(n_feats, n_classes)
        input_size  = 224
        feats_layer =  -3
        
    else:
        print('Invalid model name, exiting...')
        #exit()

    model.name           = model_name
    model.n_feats        = n_feats
    model.input_size     = input_size
    model.feats_layer    = feats_layer

    return model

In [4]:
class BaseMetaModel(nn.Module):

    def __init__(self, model):

        super().__init__()
        self.model       = model
        self.name        = self.model.name 
        self.n_feats     = self.model.n_feats
        self.input_size  = self.model.input_size
        self.feats_layer = self.model.feats_layer
        
        if 'effnet' in self.name:
            self.extract_features = self.model.extract_features

    def forward(self, img, metadata=None):
        return self.model(img)

In [5]:
class FeatureExtractor(nn.Module):
    def __init__(self, model, n_classes=8):
        super().__init__()
        self.model = model
        self.model_name = model.name

        if 'effnet' not in self.model_name:
            train_nodes, eval_nodes = get_graph_node_names(self.model)
            
            self.layer_name = eval_nodes[self.model.feats_layer]
            return_nodes    = [self.layer_name]
            #print(return_nodes)
            self.features   = create_feature_extractor(self.model, return_nodes=return_nodes)#nn.Sequential(*list(self.model.children())[:-2])

    def forward(self, x, metadata=None):
        batch_size = x.shape[0]
        if 'effnet' in self.model_name:
            x  = self.model.extract_features(x)
        elif 'vit' in self.model_name:
            x = self.features(x, metadata)[self.layer_name].permute((0, 2, 1))[:, :, :-1].reshape((batch_size, self.model.n_feats, 7, 7))
        else:
            x  = self.features(x, metadata)[self.layer_name]
            #x  = self.features(x)

        return x

In [12]:
class Passer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, feats, metadata=None):
        return feats.float()
    
    
class MetaNet(nn.Module):
    """
    Fusing Metadata and Dermoscopy Images for Skin Disease Diagnosis - https://ieeexplore.ieee.org/document/9098645
    """
    def __init__(self, n_feats, n_metadata, hidden=256):
        super(MetaNet, self).__init__()
        
        self.metaprocesser = nn.Sequential(
            nn.Conv2d(n_metadata, hidden, kernel_size=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(hidden, n_feats, kernel_size=1, padding=0),
            nn.Sigmoid()
        )

    def forward(self, feat_maps, metadata):
        x = self.metaprocesser(metadata.unsqueeze(-1).unsqueeze(-1).float())
        x = feat_maps * x
        return x
    
    
class MetaBlock(nn.Module):
    """
    Implementing the Metadata Processing Block (MetaBlock)
    """
    def __init__(self, n_feats, n_metadata):
        super().__init__()
        self.fb = nn.Sequential(nn.Linear(n_metadata, n_feats), nn.BatchNorm1d(n_feats))
        self.gb = nn.Sequential(nn.Linear(n_metadata, n_feats), nn.BatchNorm1d(n_feats))

    def forward(self, feats, metadata):
        t1 = self.fb(metadata.float()).unsqueeze(-1).unsqueeze(-1)
        t2 = self.gb(metadata.float()).unsqueeze(-1).unsqueeze(-1)
        x = torch.sigmoid(torch.tanh(feats * t1) + t2)
        return x
    
    
class FusionBlock(nn.Module):
    def __init__(self, n_feats, n_metadata, fusion_method='concat', n_reducer_block=256, p_dropout=.5):
        super().__init__()

        self.avg_pool      = nn.AvgPool2d(kernel_size=7)
        self.fusion_method = fusion_method

        if n_reducer_block > 0:
            self.reducer_block = ReducerBLock(n_reducer_block=n_reducer_block, p_dropout=p_dropout, n_feats=n_feats)
        else:
            self.reducer_block = None

        if self.fusion_method == 'metanet':
            self.fusion = MetaNet(n_feats, n_metadata)
        elif self.fusion_method == 'metablock':
            self.fusion = MetaBlock(n_feats, n_metadata)
        else:
            self.fusion = Passer()

        # Exceptions
        if n_metadata > 0 and fusion_method == None:
            raise Exception('Provide a fusion method (concat, metanet, metablock)')
        if n_metadata == 0 and fusion_method != None:
            raise Exception(f'Provide metadata for fusion method: {fusion_method}')

    def forward(self, feats, metadata):
        x = self.fusion(feats, metadata) # batch_size x n_feats x 7 x 7

        x = self.avg_pool(x)             # batch_size x n_feats x 1 x 1
        x = x.view(x.size(0), -1)        # batch_size x n_feats (flatting)

        if self.reducer_block is not None:
            x = self.reducer_block(x)    # batch_size x n_reducer_block

        if self.fusion_method == 'concat':
            x = torch.cat([x, metadata.float()], dim=1) # concatenation
        return x
    

class ReducerBLock(nn.Module):
    def __init__(self, n_reducer_block=256, p_dropout=0.5, n_feats=1024):
        super().__init__()
        self.reducer_block = nn.Sequential(
                nn.Linear(n_feats, n_reducer_block),
                nn.BatchNorm1d(n_reducer_block),
                nn.ReLU(),
                nn.Dropout(p=p_dropout)
            )
        
    def forward(self, x):
        return self.reducer_block(x)
    
    
class MetaModel(nn.Module):

    def __init__(self, model, n_classes, n_metadata=0, fusion_method='concat', n_reducer_block=256,
                 p_dropout=0.5, freeze=True):

        super().__init__()

        self.model             = model
        self.n_classes         = n_classes
        self.n_metadata        = n_metadata
        self.fusion_method     = fusion_method
        self.n_reducer_block   = n_reducer_block
        self.model_name        = model.name
        self.n_feats           = model.n_feats
        self.feature_extractor = FeatureExtractor(model, n_classes=n_classes)
        self.fusion_method     = fusion_method
        self.fusion_block      = FusionBlock(self.n_feats, n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block, p_dropout=p_dropout)

        self.n_final           = self.get_n_final()
        self.classifier        = nn.Linear(self.n_final, n_classes)

        if freeze:
            for param in self.model.parameters():
                param.requires_grad = False


    def forward(self, img, metadata):
        x = self.feature_extractor(img)       # feats:  batch_size * n_feats * 7 * 7
        x = self.fusion_block(x, metadata)    # vector: batch_size * n_reducer_block | batch_size * (n_reducer_block + n_metadata)

        return self.classifier(x)


    def get_n_final(self):
        if self.n_reducer_block > 0:
            n_model_out = self.n_reducer_block
        else:
            n_model_out = self.n_feats

        if self.fusion_method == 'concat':
            n_final = n_model_out + self.n_metadata
        else:
            n_final = n_model_out

        return n_final

# Dataset

In [13]:
training_meta = 'pad-ufes-20_parsed_folders.csv'
use_columns = ['img_id','patient_id', 'lesion_id',
       'biopsed', 'diagnostic', 'diagnostic_number', 'age', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True',
       'background_father_POMERANIA', 'background_father_GERMANY',
       'background_father_BRAZIL', 'background_father_NETHERLANDS',
       'background_father_ITALY', 'background_father_POLAND',
       'background_father_UNK', 'background_father_PORTUGAL',
       'background_father_BRASIL', 'background_father_CZECH',
       'background_father_AUSTRIA', 'background_father_SPAIN',
       'background_father_ISRAEL', 'background_mother_POMERANIA',
       'background_mother_ITALY', 'background_mother_GERMANY',
       'background_mother_BRAZIL', 'background_mother_UNK',
       'background_mother_POLAND', 'background_mother_NORWAY',
       'background_mother_PORTUGAL', 'background_mother_NETHERLANDS',
       'background_mother_FRANCE', 'background_mother_SPAIN',
       'pesticide_False', 'pesticide_True', 'gender_FEMALE', 'gender_MALE',
       'skin_cancer_history_True', 'skin_cancer_history_False',
       'cancer_history_True', 'cancer_history_False', 'has_piped_water_True',
       'has_piped_water_False', 'has_sewage_system_True',
       'has_sewage_system_False', 'fitspatrick_3.0', 'fitspatrick_1.0',
       'fitspatrick_2.0', 'fitspatrick_4.0', 'fitspatrick_5.0',
       'fitspatrick_6.0', 'region_ARM', 'region_NECK', 'region_FACE',
       'region_HAND', 'region_FOREARM', 'region_CHEST', 'region_NOSE',
       'region_THIGH', 'region_SCALP', 'region_EAR', 'region_BACK',
       'region_FOOT', 'region_ABDOMEN', 'region_LIP', 'diameter_1',
       'diameter_2', 'itch_False', 'itch_True', 'itch_UNK', 'grew_False',
       'grew_True', 'grew_UNK', 'hurt_False', 'hurt_True', 'hurt_UNK',
       'changed_False', 'changed_True', 'changed_UNK', 'bleed_False',
       'bleed_True', 'bleed_UNK', 'elevation_False', 'elevation_True',
       'elevation_UNK']

df = pd.read_csv(training_meta, usecols=use_columns)

df = df[use_columns]
df.head()

,img_id,patient_id,lesion_id,biopsed,diagnostic,diagnostic_number,age,smoke_False,smoke_True,drink_False,...,hurt_UNK,changed_False,changed_True,changed_UNK,bleed_False,bleed_True,bleed_UNK,elevation_False,elevation_True,elevation_UNK
0,PAT_1516_1765_530.png,PAT_1516,1765,False,NEV,3,8,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,PAT_46_881_939.png,PAT_46,881,True,BCC,1,55,1,0,1,...,0,0,1,0,0,1,0,0,1,0
2,PAT_1545_1867_547.png,PAT_1545,1867,False,ACK,0,77,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,PAT_1989_4061_934.png,PAT_1989,4061,False,ACK,0,75,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,PAT_1549_1882_230.png,PAT_1549,1882,False,SEK,5,53,0,0,0,...,0,1,0,0,1,0,0,0,1,0


In [14]:
open_file = open('train_idcs', "rb")
train_folds = pickle.load(open_file)
open_file.close()

open_file = open('val_idcs', "rb")
val_folds = pickle.load(open_file)
open_file.close()

open_file = open('test_idcs', "rb")
test_idcs = pickle.load(open_file)
open_file.close()

# Training

In [16]:
torch.cuda.device_count()

4

In [16]:
# model_names = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'effnetb0', 'effnetb1',
#                'effnetb2', 'effnetb3', 'effnetb4', 'effnetb5']

In [17]:
model_names = ['vit_b_32', 'vit_l_32']

In [18]:
fusion_methods = ['concat', 'metanet', 'metablock']

In [19]:
data_dir      = 'imgs'
metadata_cols = use_columns[6:]
batch_size    = 32
num_workers   = 16
input_size    = 224

train_transform = transforms.Compose([transforms.RandomResizedCrop(input_size),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transform   = transforms.Compose([transforms.Resize((input_size, input_size)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [20]:
#n_classes = len(set(train_labels))
n_epochs  = 100

lr        = 1e-3 # Learning rate
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

n_samples = df.diagnostic_number.value_counts().sort_index().values

weights = [1 - (x / sum(n_samples)) for x in n_samples]
weights = torch.FloatTensor(weights).to(device)

criterion = nn.CrossEntropyLoss(weight=weights).to(device)

saved_models_folder = 'saved_models'
saved_scores_folder = 'saved_scores'
saved_base_models_folder = 'saved_basemodels'
saved_base_scores_folder = 'saved_basescores'

cuda


In [21]:
#folds = [0, 1, 2, 3, 4]
#folds = [1, 2, 3, 4]
folds  = [0]

for fold in folds:
    
    # Dataloaders
    train_idcs = train_folds[fold]
    val_idcs   = val_folds[fold]
    train_imgs = df.loc[train_idcs, 'img_id'].values
    val_imgs   = df.loc[val_idcs, 'img_id'].values
    test_imgs  = df.loc[test_idcs, 'img_id'].values

    train_paths = [f'{os.path.join(data_dir, img)}' for img in train_imgs]
    val_paths   = [f'{os.path.join(data_dir, img)}' for img in val_imgs]
    test_paths  = [f'{os.path.join(data_dir, img)}' for img in test_imgs]
    
    train_labels = df.loc[train_idcs, 'diagnostic_number'].values
    val_labels   = df.loc[val_idcs, 'diagnostic_number'].values
    test_labels  = df.loc[test_idcs, 'diagnostic_number'].values
    
    train_metadata = df.loc[train_idcs, metadata_cols].values
    val_metadata   = df.loc[val_idcs, metadata_cols].values
    test_metadata  = df.loc[test_idcs, metadata_cols].values
    train_dataloader = get_data_loader(train_paths, train_labels, metadata=train_metadata, transform=train_transform, batch_size=batch_size, num_workers=num_workers)
    val_dataloader   = get_data_loader(val_paths, val_labels, metadata=val_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers)
    test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 
    
    # Training
    n_classes = len(set(train_labels))
    n_metadata = train_metadata.shape[1]
    for model_name in model_names:
        
        base_save_path = f'best_base_{model_name}_w_{fold}'
        base_model = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
        #base_model = get_model(model_name, n_classes=n_classes, pretrained=True).to(device)
        
        base_model.load_state_dict(torch.load(os.path.join(saved_base_models_folder, base_save_path)))
        
        for fusion_method in fusion_methods:
            print(f'{"*"*79}\n{model_name.upper()} FOLD-{fold} {fusion_method.upper()}\n{"*"*79}\n')
            if fusion_method == 'concat':
                n_reducer_block = 256
            else:
                n_reducer_block = 256
                
            save_path = f'{model_name}_{fusion_method}_{fold}'
            model     = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)
            
            optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)

            train(model, train_dataloader, val_dataloader, optimizer, scheduler, criterion, device, n_epochs,
              saved_models_folder, saved_scores_folder, save_path, printfreq=10)

            del model
            gc.collect()
            torch.cuda.empty_cache()

*******************************************************************************
VIT_B_32 FOLD-0 CONCAT
*******************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      1.0436       1.3328       0.6611      0.5653            1.0e-03       0.1
    20      0.7175       1.1328       0.7527      0.6591            1.0e-03       0.1
    30      0.4329       0.8457       0.8547      0.6733            1.0e-04       0.1
    40      0.4249       0.8111       0.8547      0.6705            1.0e-04       0.1
    50      0.4161       0.7287       0.8662      0.7585            1.0e-05       0.1
Training stopped early
-----------------------------------------------------------------------------------------
Total time [min] for 59 Epochs: 9.3
**********************************************************

# Testing

In [27]:
fusion_methods

['concat', 'metanet', 'metablock']

In [32]:
model_names = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'effnetb0', 'effnetb1',
               'effnetb2', 'effnetb3', 'effnetb4', 'effnetb5']

## Fusion methods

In [33]:
n_classes  = 6#len(set(train_labels))
n_metadata = 81
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

saved_models_folder = 'saved_models'
saved_scores_folder = 'saved_scores'

all_metrics_dict = dict()
for fold in [0, 1, 2, 3, 4]:
    print(f'{"*"*79}\n{fold}\n{"*"*79}\n')
    
    fold_dict = dict()
    for model_name in model_names:
        #print(f'{"*"*79}\n{model_name.upper()}\n{"*"*79}\n')

        base_model = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
        
        model_dict = dict()
        
        for fusion_method in fusion_methods:
            
            print(f'{"*"*79}\n{model_name.upper()} FOLD-{fold} {fusion_method.upper()}\n{"*"*79}\n')
            if fusion_method == 'concat':
                n_reducer_block = 256
            else:
                n_reducer_block = 0
            
            save_path = f'best_{model_name}_{fusion_method}_{fold}'
            
            model = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)
            model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))

            y_true, y_prob, y_pred = get_scores(model, test_dataloader, batch_size, device)
            metrics_dict = get_metrics(y_true, y_prob, y_pred)
            
            model_dict[fusion_method] = metrics_dict
            
        fold_dict[model_name] = model_dict
    all_metrics_dict[fold] = fold_dict

            #print(metrics_dict)
            #display(pd.DataFrame.from_dict(all_metrics_dict, orient='index').reset_index().rename(columns={'index': 'model'}))


*******************************************************************************
0
*******************************************************************************

*******************************************************************************
RESNET18 FOLD-0 CONCAT
*******************************************************************************

*******************************************************************************
RESNET18 FOLD-0 METANET
*******************************************************************************

*******************************************************************************
RESNET18 FOLD-0 METABLOCK
*******************************************************************************

*******************************************************************************
RESNET34 FOLD-0 CONCAT
*******************************************************************************

*******************************************************************************
RESNET34 FOLD-0 M

*******************************************************************************
RESNET50 FOLD-1 METABLOCK
*******************************************************************************

*******************************************************************************
RESNET101 FOLD-1 CONCAT
*******************************************************************************

*******************************************************************************
RESNET101 FOLD-1 METANET
*******************************************************************************

*******************************************************************************
RESNET101 FOLD-1 METABLOCK
*******************************************************************************

*******************************************************************************
RESNET152 FOLD-1 CONCAT
*******************************************************************************

*********************************************************************

*******************************************************************************
EFFNETB0 FOLD-2 METABLOCK
*******************************************************************************

Loaded pretrained weights for efficientnet-b1
*******************************************************************************
EFFNETB1 FOLD-2 CONCAT
*******************************************************************************

*******************************************************************************
EFFNETB1 FOLD-2 METANET
*******************************************************************************

*******************************************************************************
EFFNETB1 FOLD-2 METABLOCK
*******************************************************************************

Loaded pretrained weights for efficientnet-b2
*******************************************************************************
EFFNETB2 FOLD-2 CONCAT
**************************************************************

*******************************************************************************
EFFNETB3 FOLD-3 METABLOCK
*******************************************************************************

Loaded pretrained weights for efficientnet-b4
*******************************************************************************
EFFNETB4 FOLD-3 CONCAT
*******************************************************************************

*******************************************************************************
EFFNETB4 FOLD-3 METANET
*******************************************************************************

*******************************************************************************
EFFNETB4 FOLD-3 METABLOCK
*******************************************************************************

Loaded pretrained weights for efficientnet-b5
*******************************************************************************
EFFNETB5 FOLD-3 CONCAT
**************************************************************

## No metadata

In [46]:
n_classes  = 6#len(set(train_labels))
n_metadata = 81
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

saved_models_folder = 'saved_basemodels'
saved_scores_folder = 'saved_basescores'

for fold in [0, 1, 2, 3, 4]:
    print(f'{"*"*79}\n{fold}\n{"*"*79}\n')
    
    for model_name in model_names:
        #print(f'{"*"*79}\n{model_name.upper()}\n{"*"*79}\n')

        base_model = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
        
        for fusion_method in ['no_meta']:
            
            print(f'{"*"*79}\n{model_name.upper()} FOLD-{fold} {fusion_method.upper()}\n{"*"*79}\n')
            
            save_path = f'best_base_{model_name}_w_{fold}'

            model = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
            model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))

            y_true, y_prob, y_pred = get_scores(model, test_dataloader, batch_size, device)
            metrics_dict = get_metrics(y_true, y_prob, y_pred)
            
            all_metrics_dict[fold][model_name][fusion_method] = metrics_dict

*******************************************************************************
0
*******************************************************************************

*******************************************************************************
RESNET18 FOLD-0 NO_META
*******************************************************************************

*******************************************************************************
RESNET34 FOLD-0 NO_META
*******************************************************************************

*******************************************************************************
RESNET50 FOLD-0 NO_META
*******************************************************************************

*******************************************************************************
RESNET101 FOLD-0 NO_META
*******************************************************************************

*******************************************************************************
RESNET152 FOLD-0

Loaded pretrained weights for efficientnet-b5
*******************************************************************************
3
*******************************************************************************

*******************************************************************************
RESNET18 FOLD-3 NO_META
*******************************************************************************

*******************************************************************************
RESNET34 FOLD-3 NO_META
*******************************************************************************

*******************************************************************************
RESNET50 FOLD-3 NO_META
*******************************************************************************

*******************************************************************************
RESNET101 FOLD-3 NO_META
*******************************************************************************

**************************************************

## Result tables

In [87]:
import json

In [89]:
# with open('metrics.json', 'w') as outfile:
#     json.dump(all_metrics_dict, outfile)

In [98]:
# with open('metrics.json') as json_file:
#     data = json.load(json_file)
#     print(data)

In [64]:
folds = [0, 1, 2, 3, 4]

In [47]:
all_metrics_dict

{0: {'resnet18': {'concat': {'precision': 0.6789797326932954,
    'recall': 0.7048054919908466,
    'f1-score': 0.6842040721485715,
    'support': 437,
    'accuracy': 0.7048054919908466,
    'balanced_accuracy': 0.5996484636639171,
    'auc': 0.9121420315256327},
   'metanet': {'precision': 0.671518673928943,
    'recall': 0.6887871853546911,
    'f1-score': 0.6718132010314587,
    'support': 437,
    'accuracy': 0.6887871853546911,
    'balanced_accuracy': 0.5609743124944172,
    'auc': 0.890590503922404},
   'metablock': {'precision': 0.6789163261551344,
    'recall': 0.7070938215102975,
    'f1-score': 0.6636090291995692,
    'support': 437,
    'accuracy': 0.7070938215102975,
    'balanced_accuracy': 0.4830338202298723,
    'auc': 0.9143656840931126},
   'no_meta': {'precision': 0.6649992479288678,
    'recall': 0.6910755148741419,
    'f1-score': 0.6701864433408058,
    'support': 437,
    'accuracy': 0.6910755148741419,
    'balanced_accuracy': 0.5775953132011243,
    'auc': 0.8

In [63]:
all_metrics_dict[1]['resnet18']#['concat']

{'concat': {'precision': 0.7114323003569345,
  'recall': 0.7116704805491991,
  'f1-score': 0.7088388431731882,
  'support': 437,
  'accuracy': 0.7116704805491991,
  'balanced_accuracy': 0.6547728569817577,
  'auc': 0.9127869211086815},
 'metanet': {'precision': 0.6442943811373407,
  'recall': 0.6979405034324943,
  'f1-score': 0.668869894041926,
  'support': 437,
  'accuracy': 0.6979405034324943,
  'balanced_accuracy': 0.576493473545123,
  'auc': 0.8944482811060149},
 'metablock': {'precision': 0.7265842348349891,
  'recall': 0.7185354691075515,
  'f1-score': 0.6803906903756777,
  'support': 437,
  'accuracy': 0.7185354691075515,
  'balanced_accuracy': 0.49562738088631014,
  'auc': 0.9126934212241468},
 'no_meta': {'precision': 0.6220703957640973,
  'recall': 0.6430205949656751,
  'f1-score': 0.6255844498928941,
  'support': 437,
  'accuracy': 0.6430205949656751,
  'balanced_accuracy': 0.5302866997036145,
  'auc': 0.8831806769495684}}

In [68]:
pd.DataFrame.from_dict(all_metrics_dict[4]['resnet18'], orient='index')

,precision,recall,f1-score,support,accuracy,balanced_accuracy,auc
concat,0.715620,0.709382,0.708166,437,0.709382,0.638441,0.905237
metanet,0.669207,0.691076,0.672161,437,0.691076,0.577000,0.892889
metablock,0.734417,0.716247,0.674960,437,0.716247,0.490563,0.917088
no_meta,0.662082,0.681922,0.669328,437,0.681922,0.581076,0.877825


In [83]:
model_mean_dict = dict()
model_std_dict  = dict()

for model_name in model_names:
    print(model_name)
    model_df = pd.DataFrame()
    
    for fold in folds:
        model_df = pd.concat([model_df, pd.DataFrame.from_dict(all_metrics_dict[fold]['resnet18'], orient='index')])
    mean_df = model_df.reset_index().groupby('index').mean()
    std_df  = model_df.reset_index().groupby('index').std()
    
    model_mean_dict[model_name] = mean_df.to_dict()
    model_std_dict[model_name]  = std_df.to_dict()

resnet18
resnet34
resnet50
resnet101
resnet152
effnetb0
effnetb1
effnetb2
effnetb3
effnetb4
effnetb5


In [84]:
model_mean_dict

{'resnet18': {'precision': {'concat': 0.6986986603146164,
   'metablock': 0.726431313201741,
   'metanet': 0.6626244865892965,
   'no_meta': 0.648124206448248},
  'recall': {'concat': 0.7157894736842105,
   'metablock': 0.7189931350114417,
   'metanet': 0.6860411899313501,
   'no_meta': 0.662700228832952},
  'f1-score': {'concat': 0.7031260176344069,
   'metablock': 0.6782485364174855,
   'metanet': 0.6675085993695618,
   'no_meta': 0.6496664776575981},
  'support': {'concat': 437.0,
   'metablock': 437.0,
   'metanet': 437.0,
   'no_meta': 437.0},
  'accuracy': {'concat': 0.7157894736842105,
   'metablock': 0.7189931350114417,
   'metanet': 0.6860411899313501,
   'no_meta': 0.662700228832952},
  'balanced_accuracy': {'concat': 0.6307864270710593,
   'metablock': 0.49848653272550625,
   'metanet': 0.5773205297134795,
   'no_meta': 0.5666259346629783},
  'auc': {'concat': 0.9116731273606172,
   'metablock': 0.917094681158251,
   'metanet': 0.8970026884618868,
   'no_meta': 0.88253745651

In [86]:
metrics = ['precision', 'recall', 'balanced_accuracy', 'auc']

str_model_dict = {model_name: {metric: f'${model_mean_dict[model_name][metric]:.2f} \\pm {model_std_dict[model_name][metric]:.2f}$' for metric in metrics} for model_name in model_names}

TypeError: unsupported format string passed to dict.__format__

In [82]:
# mean_df = model_df.reset_index().groupby('index').mean()
# std_df  = model_df.reset_index().groupby('index').std()

# mean_df.to_dict()

{'precision': {'concat': 0.6986986603146164,
  'metablock': 0.726431313201741,
  'metanet': 0.6626244865892965,
  'no_meta': 0.648124206448248},
 'recall': {'concat': 0.7157894736842105,
  'metablock': 0.7189931350114417,
  'metanet': 0.6860411899313501,
  'no_meta': 0.662700228832952},
 'f1-score': {'concat': 0.7031260176344069,
  'metablock': 0.6782485364174855,
  'metanet': 0.6675085993695618,
  'no_meta': 0.6496664776575981},
 'support': {'concat': 437.0,
  'metablock': 437.0,
  'metanet': 437.0,
  'no_meta': 437.0},
 'accuracy': {'concat': 0.7157894736842105,
  'metablock': 0.7189931350114417,
  'metanet': 0.6860411899313501,
  'no_meta': 0.662700228832952},
 'balanced_accuracy': {'concat': 0.6307864270710593,
  'metablock': 0.49848653272550625,
  'metanet': 0.5773205297134795,
  'no_meta': 0.5666259346629783},
 'auc': {'concat': 0.9116731273606172,
  'metablock': 0.917094681158251,
  'metanet': 0.8970026884618868,
  'no_meta': 0.882537456510066}}

In [65]:
{fusion_method: {fold: all_metrics_dict[fold]['resnet18'][fusion_method] for fold in folds} for fusion_method in fusion_methods}

{'concat': {0: {'precision': 0.6789797326932954,
   'recall': 0.7048054919908466,
   'f1-score': 0.6842040721485715,
   'support': 437,
   'accuracy': 0.7048054919908466,
   'balanced_accuracy': 0.5996484636639171,
   'auc': 0.9121420315256327},
  1: {'precision': 0.7114323003569345,
   'recall': 0.7116704805491991,
   'f1-score': 0.7088388431731882,
   'support': 437,
   'accuracy': 0.7116704805491991,
   'balanced_accuracy': 0.6547728569817577,
   'auc': 0.9127869211086815},
  2: {'precision': 0.6758186741894578,
   'recall': 0.7391304347826086,
   'f1-score': 0.7050089286717621,
   'support': 437,
   'accuracy': 0.7391304347826086,
   'balanced_accuracy': 0.6184206814049378,
   'auc': 0.9141776962260074},
  3: {'precision': 0.7116424935452157,
   'recall': 0.7139588100686499,
   'f1-score': 0.70941185926856,
   'support': 437,
   'accuracy': 0.7139588100686499,
   'balanced_accuracy': 0.6426493728868423,
   'auc': 0.914022289180107},
  4: {'precision': 0.7156201007881784,
   'recall

In [66]:
pd.DataFrame.from_dict({fusion_method: {fold: all_metrics_dict[fold]['resnet18'][fusion_method] for fold in folds} for fusion_method in fusion_methods}, orient='index')

,0,1,2,3,4
concat,"{'precision': 0.6789797326932954, 'recall': 0....","{'precision': 0.7114323003569345, 'recall': 0....","{'precision': 0.6758186741894578, 'recall': 0....","{'precision': 0.7116424935452157, 'recall': 0....","{'precision': 0.7156201007881784, 'recall': 0...."
metanet,"{'precision': 0.671518673928943, 'recall': 0.6...","{'precision': 0.6442943811373407, 'recall': 0....","{'precision': 0.6716168304203098, 'recall': 0....","{'precision': 0.6564855835618496, 'recall': 0....","{'precision': 0.6692069638980395, 'recall': 0...."
metablock,"{'precision': 0.6789163261551344, 'recall': 0....","{'precision': 0.7265842348349891, 'recall': 0....","{'precision': 0.7495363941322605, 'recall': 0....","{'precision': 0.742702216386427, 'recall': 0.7...","{'precision': 0.7344173944998937, 'recall': 0...."


In [ ]:
pd.DataFrame.from_dict(all_metrics_dict[4]['resnet18'], orient='index')

In [ ]:
for fusion_method in fusion_methods:
    
    model_mean_dict = dict()
    model_std_dict  = dict()

    for model_name in all_metrics_dict.keys():
        model_df = pd.DataFrame.from_dict(all_metrics_dict[model_name], orient='index')

        model_mean_dict[model_name] = model_df.mean().to_dict()
        model_std_dict[model_name]  = model_df.std().to_dict()
    for fold in folds:
        for model_name in model_names:
            

In [ ]:
for model_name in model_names:
    model_mean_dict = dict()
    model_std_dict  = dict()
    
    model_df = pd.DataFrame.from_dict(all_metrics_dict[fold][model_name], orient='index')
    for fold in folds:
        